In [ ]:
import os, sys 
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/mnt


In [ ]:
import glob
import re
import math
import pandas as pd

In [23]:
label_path = '/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/데이터 라벨링/괴도키즈_데이터_라벨링 - UCF-Crime Data.csv'

In [ ]:
abuse_path = '/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/Data/UCF-Crime/Anomaly-Videos-Part-1_Abuse/'
assault_path = '/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/Data/UCF-Crime/Anomaly-Videos-Part-1_Assault/'
save_path = '/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/Data/UCF-Crime/UCF-Crime_Dataset/'

# Crop Fight Subclip

In [40]:
label = pd.read_csv(label_path)

In [ ]:
label.columns = ['filename','body','tools','information','seen_by']

In [ ]:
label.drop('seen_by',axis=1, inplace=True)

In [ ]:
label = label.fillna('na')

In [ ]:
label.head()

,filename,body,tools,information
0,Abuse001,"(0:08,0:11)(0:33,0:36)(1:16,1:19)",na,3번 반복이 되는 영상
1,Abuse002,na,na,x (차량)
2,Abuse003,"(0:30,0:33)(0:34,0:36)(1:07,1:15)(1:47,1:53)\r",na,na
3,Abuse004,"(0:30,0:34)",na,폭력이 받는 대상이 frame안에 있을때만 기록
4,Abuse005,"(0:24,0:29)\r",na,na


In [ ]:
def isNaN(string):
    return string == 'na'

In [ ]:
isNaN(label.iloc[0].body), isNaN(label.iloc[0].tools)

(False, True)

In [ ]:
label = label[label['information'].map(lambda info: not str(info).startswith('x'))]                 # information이 x로 시작하는 row 제거
label = label[label['filename'].map(lambda info: not isNaN(info))]                                  # filename 없는 row 제거
label['body'] = label['body'].map(lambda x: str(x).strip())                                         # body column 속 whitespace 제거
label = label[~((label['body'].map(lambda x:isNaN(x))) & (label['tools'].map(lambda x:isNaN(x))))]  # body와 tools 모두 nan인 row 제거

In [ ]:
label

,filename,body,tools,information
0,Abuse001,"(0:08,0:11)(0:33,0:36)(1:16,1:19)",na,3번 반복이 되는 영상
2,Abuse003,"(0:30,0:33)(0:34,0:36)(1:07,1:15)(1:47,1:53)",na,na
3,Abuse004,"(0:30,0:34)",na,폭력이 받는 대상이 frame안에 있을때만 기록
4,Abuse005,"(0:24,0:29)",na,na
5,Abuse006,"(0:10,1:00)",na,일대다 폭력이 오래 지속됨
...,...,...,...,...
98,Assault048,"(0:07, 0:10)",na,프레임 밖
99,Assault049,"(0:12,0:18)(0:20,0:29)",na,같은 영상 반복됨
100,Assault050,"(0:03,0:20)",na,na
101,Assault051,"(0:06,0:07)","(0:09,0:15)",두번째 폭력 다구리


In [ ]:
for video in label.itertuples():
    if isNaN(video.body) and isNaN(video.tools): 
        print(video.Index,video.filename,video.body, video.tools, video.information)

In [ ]:
for video in label.itertuples():
    print('---------------------------- ',video.Index)
    print(video.body, video.tools)
    if not isNaN(video.body):
        print(re.findall(r'\(([\d ]+:[\d ]+),([\d ]+:[\d ]+)\)',video.body))
    if not isNaN(video.tools):
        print(re.findall(r'\(([\d ]+:[\d ]+),([\d ]+:[\d ]+)\)',video.tools))


----------------------------  0
(0:08,0:11)(0:33,0:36)(1:16,1:19) na
[('0:08', '0:11'), ('0:33', '0:36'), ('1:16', '1:19')]
----------------------------  2
(0:30,0:33)(0:34,0:36)(1:07,1:15)(1:47,1:53) na
[('0:30', '0:33'), ('0:34', '0:36'), ('1:07', '1:15'), ('1:47', '1:53')]
----------------------------  3
(0:30,0:34) na
[('0:30', '0:34')]
----------------------------  4
(0:24,0:29) na
[('0:24', '0:29')]
----------------------------  5
(0:10,1:00) na
[('0:10', '1:00')]
----------------------------  6
(0:02,0:05) na
[('0:02', '0:05')]
----------------------------  7
(2:27,2:30) na
[('2:27', '2:30')]
----------------------------  8
(0:08,0:11)(0:22,0:25) na
[('0:08', '0:11'), ('0:22', '0:25')]
----------------------------  10
(0:11,0:17)(0:31,0:38)(0:45,0:52)(1:05,1:08)(1:20,1:27)(1:48,1:50) na
[('0:11', '0:17'), ('0:31', '0:38'), ('0:45', '0:52'), ('1:05', '1:08'), ('1:20', '1:27'), ('1:48', '1:50')]
----------------------------  11
(0:10,0:14)(0:16,0:18)(1:00,1:06)(1:14,1:16)(1:41,1:4

In [ ]:
def find_video(path, filename):
    file_list = os.listdir(path)
    filefound = list(filter(lambda x:x.startswith(filename), file_list))
    print(filefound)
    if len(filefound) == 0:
        print('{} / file not found in path / {}'.format(filename,path))
        return -1
    if len(filefound) > 1:
        print(str(filefound) , ' / more than one file matched')
        return -1
    return path + filefound[0]

In [ ]:
from moviepy.tools import subprocess_call
from moviepy.config import get_setting

def ffmpeg_extract_subclip(filename, t1, t2, targetname=None):
    """ Makes a new video file playing video file ``filename`` between
    the times ``t1`` and ``t2``. """
    name, ext = os.path.splitext(filename)
    if not targetname:
        T1, T2 = [int(1000*t) for t in [t1, t2]]
        targetname = "%sSUB%d_%d.%s" % (name, T1, T2, ext)

    cmd = [get_setting("FFMPEG_BINARY"),"-y",
           "-ss", "%0.2f"%t1,
           "-i", filename,
           "-t", "%0.2f"%(t2-t1),
           "-vcodec", "copy", "-acodec", "copy", targetname]

    subprocess_call(cmd)

In [ ]:
def ucf_data_subclip_extract(abuse_path, assault_path, save_path, label, dst_name):
    out_path = list()
    for video in label.itertuples():
        pattern = r'\(([\d ]+:[\d ]+),([\d ]+:[\d ]+)\)'
        index = 1
        if video.filename.startswith('Abuse'):
            source = find_video(abuse_path, video.filename)
        elif video.filename.startswith('Assault'):
            source = find_video(assault_path, video.filename)
        else:
            break

        if source == -1:
            break

        print(source)
        for start, end in re.findall(pattern,video.body):
            start = float(start.split(':')[0]) * 60 + float(start.split(':')[1])
            end = float(end.split(':')[0]) * 60 + float(end.split(':')[1])
            print(start, end)
            assert end > start
            dst = save_path + video.filename + dst_name + '.mp4'
            out_path.append(dst)

            ffmpeg_extract_subclip(source, start, end, targetname=dst)
            index += 1

        for start, end in re.findall(pattern,video.tools):
            start = float(start.split(':')[0]) * 60 + float(start.split(':')[1])
            end = float(end.split(':')[0]) * 60 + float(end.split(':')[1])
            print(start, end)
            assert end > start
            dst = save_path + video.filename + dst_name + '.mp4' 
            out_path.append(dst)

            ffmpeg_extract_subclip(source, start, end, targetname=dst)
            index += 1
        
        print('Sucessfully Finished - ',video.filename)
    return out_path

ucf_subclip_list = ucf_data_subclip_extract(abuse_path, assault_path, save_path, label, '_'.join(['',str(index)]))

['Abuse001_x264.mp4']
/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/Data/UCF-Crime/Anomaly-Videos-Part-1_Abuse/Abuse001_x264.mp4
8.0 11.0
33.0 36.0
76.0 79.0
Sucessfully Finished -  Abuse001
['Abuse003_x264.mp4']
/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/Data/UCF-Crime/Anomaly-Videos-Part-1_Abuse/Abuse003_x264.mp4
30.0 33.0
34.0 36.0
67.0 75.0
107.0 113.0
Sucessfully Finished -  Abuse003
['Abuse004_x264.mp4']
/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/Data/UCF-Crime/Anomaly-Videos-Part-1_Abuse/Abuse004_x264.mp4
30.0 34.0
Sucessfully Finished -  Abuse004
['Abuse005_x264.mp4']
/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/Data/UCF-Crime/Anomaly-Videos-Part-1_Abuse/Abuse005_x264.mp4
24.0 29.0
Sucessfully Finished -  Abuse005
['Abuse006_x264.mp4']
/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/Data/UCF-Crime/Anomaly-Videos-Part-1_Abuse/Abuse006_x264.mp4
10.0 60.0
Sucessfully Finished -  Abuse006
['Abuse007_x264.mp4']
/c

# Crop Non-Fight Subclip

In [41]:
non_fight_label_path = '/content/mnt/Shareddrives/2021청년인재_고려대과정_10조/데이터 라벨링/괴도키즈_데이터_라벨링 - UCF-Crime_nonfight_Data.csv'
non_fight_label = pd.read_csv(non_fight_label_path)

In [42]:
non_fight_label.columns = ['filename', 'nonfight', 'maybe', 'nonfightinformation','body','tools','information']

In [43]:
non_fight_label.head()

,filename,nonfight,maybe,nonfightinformation,body,tools,information
0,Abuse001,"(0:00,0:08)(0:25,0:33)","(0:11,0:25)",NaN,"(0:08,0:11)(0:33,0:36)(1:16,1:19)",NaN,3번 반복이 되는 영상
1,Abuse002,x,x,x,NaN,NaN,x (차량)
2,Abuse003,"(0:00,0:30)(0:36,1:07)(1:15,1:47)",x,NaN,"(0:30,0:33)(0:34,0:36)(1:07,1:15)(1:47,1:53)\n",NaN,NaN
3,Abuse004,"(0:00,0:16)(0:18,0:28)(1:08,1:30)(8:50,9:19)",x,NaN,"(0:30,0:34)",NaN,폭력이 받는 대상이 frame안에 있을때만 기록
4,Abuse005,"(0:00,0:21)",x,NaN,"(0:24,0:29)\n",NaN,NaN


In [44]:
def isNaN(string):
    return string == 'na' or string == 'x'

In [45]:
non_fight_label.drop(['body','tools','information'], axis=1, inplace=True)

In [47]:
non_fight_label = non_fight_label.fillna('na')

In [48]:
non_fight_label = non_fight_label[non_fight_label['nonfightinformation'].map(lambda info: not str(info).startswith('x'))]                       # information이 x로 시작하는 row 제거
non_fight_label = non_fight_label[non_fight_label['filename'].map(lambda info: not isNaN(info))]                                                # filename 없는 row 제거   
non_fight_label['nonfight'] = non_fight_label['nonfight'].map(lambda x: str(x).strip())                                                         # nonfight column 속 whitespace 제거
non_fight_label = non_fight_label[~((non_fight_label['nonfight'].map(lambda x:isNaN(x))) & (non_fight_label['maybe'].map(lambda x:isNaN(x))))]  # body와 tools 모두 nan인 row 제거
non_fight_label['nonfight'] = non_fight_label['nonfight'].map(lambda x: re.sub(';', ':', x))                                                    # 오타 제거
non_fight_label['maybe'] = non_fight_label['maybe'].map(lambda x: re.sub(';', ':', x))

In [50]:
for video in non_fight_label.itertuples():
    #print(video.body, video.tools)

    print('---------------------------- ',video.Index)
    print(video.nonfight, video.maybe)
    if not isNaN(video.nonfight):
        print(re.findall(r'\(([\d ]+:[\d ]+),([\d ]+:[\d ]+)\)',video.nonfight))
    if not isNaN(video.maybe):
        print(re.findall(r'\(([\d ]+:[\d ]+),([\d ]+:[\d ]+)\)',video.maybe))

----------------------------  0
(0:00,0:08)(0:25,0:33) (0:11,0:25)
[('0:00', '0:08'), ('0:25', '0:33')]
[('0:11', '0:25')]
----------------------------  2
(0:00,0:30)(0:36,1:07)(1:15,1:47) x
[('0:00', '0:30'), ('0:36', '1:07'), ('1:15', '1:47')]
----------------------------  3
(0:00,0:16)(0:18,0:28)(1:08,1:30)(8:50,9:19) x
[('0:00', '0:16'), ('0:18', '0:28'), ('1:08', '1:30'), ('8:50', '9:19')]
----------------------------  4
(0:00,0:21) x
[('0:00', '0:21')]
----------------------------  5
(0:00,0:09) (1:01,1:40)
[('0:00', '0:09')]
[('1:01', '1:40')]
----------------------------  6
(0:10,0:38) x
[('0:10', '0:38')]
----------------------------  7
(0:00,0:10)(0:53,2:25) (0:10,0:53)
[('0:00', '0:10'), ('0:53', '2:25')]
[('0:10', '0:53')]
----------------------------  10
(0:38,0:43)(1:36:1:40) (0:20,0:26)
[('0:38', '0:43')]
[('0:20', '0:26')]
----------------------------  11
(0:00,0:09) (0:19,0:55)(1:18,1:30)(1:43,2:51)
[('0:00', '0:09')]
[('0:19', '0:55'), ('1:18', '1:30'), ('1:43', '2:51

In [ ]:
ucf_data_subclip_extract(abuse_path, assault_path, save_path, non_fight_label, '_'.join(['','nofi',str(index)]))